In [1]:
import pandas as pd
import pulp as plp
import numpy as np

In [2]:
#Definisi constrain
#1 customer harus di handle oleh 1 profcoll
cust_df  = pd.DataFrame({'customer': ['customer1', 'customer2', 'customer3'],
                             'resource':[1, 1, 1]
                             })
#Jumlah available resource dari tiap profcoll
profcoll_df  = pd.DataFrame({'profcoll': ['profcoll1', 'profcoll2'],
                             'resource':[2, 1]
                             })

#Estimasi biaya dari masing2 profcoll untuk tiap customer 
cost = np.array([[10000,15000], [5000,10000], [12500,12500]])

cost_df = pd.DataFrame(cost)
cost_df.columns=profcoll_df.profcoll.values
cost_df.insert(0, "Allocation", cust_df['customer'])
cost_df.set_index('Allocation',inplace=True)
cost_df

,profcoll1,profcoll2
Allocation,,
customer1,10000,15000
customer2,5000,10000
customer3,12500,12500


In [12]:
cust_df

,customer,resource
0,customer1,1
1,customer2,1
2,customer3,1


In [3]:
model = plp.LpProblem("Resource_allocation_prob", plp.LpMinimize)

In [4]:
no_of_customer = cust_df.shape[0]
no_of_profcoll = profcoll_df.shape[0]
x_vars_list = []
for l in range(1,no_of_customer+1):
    for w in range(1,no_of_profcoll+1):
        temp = str(l)+str(w)
        x_vars_list.append(temp)
x_vars = plp.LpVariable.matrix("x", x_vars_list, cat = "Integer", lowBound = 0)
final_allocation = np.array(x_vars).reshape(3,2)
print(final_allocation)
res_equation = plp.lpSum(final_allocation*cost)
model +=  res_equation

[[x_11 x_12]
 [x_21 x_22]
 [x_31 x_32]]


In [5]:
for c1 in range(no_of_customer):
    model += plp.lpSum(final_allocation[c1][p1] for p1 in range(no_of_profcoll)) == cust_df['resource'].tolist()[c1]
for p2 in range(no_of_profcoll):
    model += plp.lpSum(final_allocation[c2][p2] for c2 in range(no_of_customer)) <= profcoll_df['resource'].tolist()[p2]

In [6]:
print(model)

Resource_allocation_prob:
MINIMIZE
10000*x_11 + 15000*x_12 + 5000*x_21 + 10000*x_22 + 12500*x_31 + 12500*x_32 + 0
SUBJECT TO
_C1: x_11 + x_12 = 1

_C2: x_21 + x_22 = 1

_C3: x_31 + x_32 = 1

_C4: x_11 + x_21 + x_31 <= 2

_C5: x_12 + x_22 + x_32 <= 1

VARIABLES
0 <= x_11 Integer
0 <= x_12 Integer
0 <= x_21 Integer
0 <= x_22 Integer
0 <= x_31 Integer
0 <= x_32 Integer



In [7]:
model.solve()
status = plp.LpStatus[model.status]
print(status)

Optimal


In [11]:
print("Optimal overall cost: ",str(plp.value(model.objective)))
for each in model.variables():
    print("Match ", each, ": "+str(each.value()))

Optimal overall cost:  27500.0
Match  x_11 : 1.0
Match  x_12 : 0.0
Match  x_21 : 1.0
Match  x_22 : 0.0
Match  x_31 : 0.0
Match  x_32 : 1.0


In [9]:
x=[]
for v in model.variables ():
    x.append(v.varValue)
result = np.reshape(x, (-1, no_of_profcoll))
result_df = pd.DataFrame(result)
result_df.columns=profcoll_df.profcoll.values
result_df.insert(0, "Allocation", cust_df['customer'])
result_df.set_index('Allocation',inplace=True)
result_df

,profcoll1,profcoll2
Allocation,,
customer1,1.0,0.0
customer2,1.0,0.0
customer3,0.0,1.0


In [10]:
cost_df

,profcoll1,profcoll2
Allocation,,
customer1,10000,15000
customer2,5000,10000
customer3,12500,12500
